In [ ]:
county_2015["income"] = county_2015["B19013_001E"].astype(float)
county_2015["population"] = county_2015["B01001_001E"].astype(float) 

nquantiles = 4

labels = ["quantile " + str(var) for var in range(1,nquantiles + 1)]

inc_q = pd.qcut(county_2015["income"], nquantiles, labels = labels)

county_2015.groupby(inc_q).mean()


county_2015.pivot_table(["income", "population"], index = inc_q)

In [ ]:
labels = ["quantile " + str(var) for var in range(0,4)]
labels

In [ ]:
county_2015.pivot_table(["income", "population"], index = inc_q).plot(kind = 'barh')
plt.show()

In [ ]:
code = ("B19301_001E") # This is per capita income

county_2015 = pd.DataFrame(c.acs5.get(code, 
                                         {'for': 'zip code tabulation area:*'}, year=2014))
county_2015.tail(10)

In [ ]:
code = ("B19326_006E") # This is median income, worked full time, women

county_2015 = pd.DataFrame(c.acs5.get(code, 
                                         {'for': 'state:*'}, year=2014))
county_2015.head()

code = ("B19326_003E") # This is median income, worked full time, men

county_2015 = pd.DataFrame(c.acs5.get(code, 
                                         {'for': 'state:*'}, year=2014))

county_2015.head()

In [ ]:
from bokeh.io import show
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LogColorMapper
)
from bokeh.palettes import Viridis6 as palette
from bokeh.plotting import figure

#import bokeh.sampledata

#bokeh.sampledata.download()

from bokeh.sampledata.us_counties import data as counties
from bokeh.sampledata.unemployment import data as unemployment

palette.reverse()

counties = {
    code: county for code, county in counties.items() if county["state"] == "ca"
    
}

county_xs = [county["lons"] for county in counties.values()]
county_ys = [county["lats"] for county in counties.values()]


county_names = [county['name'] for county in counties.values()]
county_rates = [unemployment[county_id] for county_id in counties]
color_mapper = LogColorMapper(palette=palette)

source = ColumnDataSource(data=dict(
    x=county_xs,
    y=county_ys,
    name=county_names,
    rate=county_rates,
))

TOOLS = "pan,wheel_zoom,reset,hover,save"

p = figure(
    title="Texas Unemployment, 2009", tools=TOOLS,
    x_axis_location=None, y_axis_location=None
)
p.grid.grid_line_color = None

p.patches('x', 'y', source=source,
          fill_color={'field': 'rate', 'transform': color_mapper},
          fill_alpha=0.7, line_color="white", line_width=0.5)

hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [
    ("Name", "@name"),
    ("Unemployment rate)", "@rate%"),
    ("(Long, Lat)", "($x, $y)"),
]

show(p)